# 付録 10.4.3: チャート、グラフ、スライドデッキの操作
Claudeは、チャート、グラフ、そして広範なスライドデッキを扱う能力に優れています。使用ケースに応じて、活用したいヒントやコツがいくつかあります。このレシピでは、これらの資料を使用する際のClaudeの一般的なパターンを示します。

## チャートとグラフ
ほとんどの場合、`claude`を使用してチャートやグラフを扱うのは簡単です。これらを取り込んで`Claude`に渡す方法や、結果を改善するための一般的なヒントを見ていきましょう。

### インジェクションとClaude APIの呼び出し
Claudeのチャートやグラフを渡す最良の方法は、その視覚機能を活用することです。つまり、チャートやグラフの画像と、それに関するテキストの質問をClaudeに与えます。すべてのバージョンのClaudeは画像を受け入れることができますが、データが多い画像タスクにはSonnetとOpusが推奨モデルです。Sonnetを使用して始めましょう。

In [ ]:
pip install -qUr requirements.txt

In [ ]:
import boto3
import json
import base64
from datetime import datetime
from IPython.display import Image
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

# modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

print(f'Using modelId: {modelId}')
print('Using region: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

In [ ]:
def get_completion(messages):
    converse_api_params = {
        "modelId": modelId,
        "messages": messages,
    }
    response = bedrock_client.converse(**converse_api_params)
    # レスポンスから生成されたテキストコンテンツを抽出します
    return response['output']['message']['content'][0]['text']

In [ ]:
# まず、画像が必要です。cvna_2021_annual_report_image.pngにある.png画像を使用します。
# 画像を読み込み、base64としてエンコードします。
with open("./images/reading_charts_graphs/cvna_2021_annual_report_image.png", "rb") as f:
    image_file = f.read()

# 自分たちでも画像を見てみましょう
Image(filename='./images/reading_charts_graphs/cvna_2021_annual_report_image.png')

この画像をモデルに渡し、シンプルな質問をする方法を見てみましょう。

In [ ]:
messages = [
    {
        "role": 'user',
        "content": [
            {"text": "この画像には何が含まれていますか？一文で答えてください。"},
            {"image": {
                "format": 'png',
                "source": {"bytes": image_file }
                },
            }
        ]
    }
]

print(get_completion(messages))

それはとても良いですね！では、さらに有用な質問をしてみましょう。

In [ ]:
questions = [
    "What was CVNA revenue in 2020?",
    "How many additional markets has Carvana added since 2014?",
    "What was 2016 revenue per retail unit sold?"
]

for index, question in enumerate(questions):
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": "この画像には何が含まれていますか？一文で答えてください。"},
                {"image": {
                    "format": 'png',
                    "source": {"bytes": image_file }
                    },
                }
            ]
        }
    ]
    
    print(f"\n----------質問 {index+1}----------")
    print(get_completion(messages))

ご覧のとおり、Claudeはチャートやグラフに関するかなり詳細な質問に答えることができます。しかし、最大限に活用するためのいくつかのヒントやコツがあります。
- 時々、Claudeの算数能力が邪魔をすることがあります。上記の3番目の質問をサンプリングすると、算数を間違えて不正確な最終回答を出力することがあることに気付くでしょう。このような間違いを避けるために、Claudeに計算機ツールを提供することを検討してください。
- 非常に複雑なチャートやグラフの場合、Claudeに「画像に見えるすべてのデータポイントを最初に説明してください」と尋ねることで、従来のChain of Thoughtで見られるような改善を引き出すことができます。
- Claudeは、情報を伝えるために多くの色に依存するチャート、例えば多くのグループを持つグループ化された棒グラフに苦労することがあります。最初にグラフの色をHEXコードを使って特定するようにClaudeに頼むことで、その精度を向上させることができます。

## スライドデッキ
Claudeがチャートやグラフの達人であることがわかった今、チャートやグラフの真のホームであるスライドデッキにそれを拡張するのは理にかなっています！

スライドは、金融サービスを含む多くの分野にとって重要な情報源です。スライドデッキからテキストを抽出するためにPyPDFのようなパッケージを使用することは*できます*が、チャートやグラフが多い性質のため、モデルが実際に必要な情報にアクセスするのが難しくなるため、これはあまり良い選択肢ではありません。その結果、Visionは素晴らしい代替手段となることがあります。このセクションでは、vision Claudeを使用してスライドデッキをレビューする方法と、このアプローチの一般的な落とし穴に対処する方法について説明します。

典型的なスライドデッキを`claude`に取り込む最良の方法は、それをPDFとしてダウンロードし、次に各PDFページを画像に変換することです。これを達成する方法は以下の通りです。

In [ ]:
#%pip install PyMuPDF

In [ ]:
from PIL import Image
import io
import fitz

def pdf_to_pngs(pdf_path, quality=75, max_size=(1024, 1024)):
    """
    PDFファイルをPNG画像のリストに変換します。

    Args:
        pdf_path (str): PDFファイルへのパス。
        quality (int, optional): 出力PNG画像の品質（デフォルトは75）。
        max_size (tuple, optional): 出力画像の最大サイズ（デフォルトは(1024, 1024)）。

    Returns:
        list: バイトとしてのPNG画像のリスト。
    """
    # PDFファイルを開く
    doc = fitz.open(pdf_path)
    pdf_to_png_images = []

    # PDFの各ページを反復処理
    for page_num in range(doc.page_count):
        # ページを読み込む
        page = doc.load_page(page_num)

        # ページをPNG画像としてレンダリング
        pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))

        # PNG画像を保存
        output_path = f"./images/reading_charts_graphs/slides/page_{page_num+1}.png"
        pix.save(output_path)

        # PILを使用して保存した画像を開く
        image = Image.open(output_path)

        # 画像が最大サイズを超える場合はリサイズ
        if image.size[0] > max_size[0] or image.size[1] > max_size[1]:
            image.thumbnail(max_size, Image.Resampling.LANCZOS)

        # PIL画像をバイトに変換
        image_data = io.BytesIO()
        image.save(image_data, format='PNG', optimize=True, quality=quality)
        image_data.seek(0)
        pdf_to_png_image = image_data.getvalue()

        # PNG画像のバイトをリストに追加
        pdf_to_png_images.append(pdf_to_png_image)

    # PDFドキュメントを閉じる
    doc.close()

    return pdf_to_png_images

# PDFファイルへのパスを指定
pdf_path = './images/reading_charts_graphs/twilio_q4_2023.pdf'
# pdf_to_pngs関数を呼び出してPDFをPNG画像に変換
pdf_pngs = pdf_to_pngs(pdf_path)

In [ ]:
# 最初の20枚の画像を一度にClaudeに渡し、デッキについて質問します。
# なぜ20枚なのか？現在、Anthropic APIは最大20枚の画像を渡すことしか許可していません。
# この数は時間とともに増える可能性がありますが、後でこのレシピで管理するための役立つヒントがあります。

content = [{"image": {"format": 'png', "source": {"bytes": pdf_png}}} for pdf_png in pdf_pngs[:20]]

question = "2023会計年度のTwilioの前年比収益成長率は何でしたか？"
#question = "非GAAPの粗利益率は何でしたか？"

# 画像に質問を追加します
content.append({"text": question})

messages = [
    {
        "role": 'user',
        "content": content
    }
]

print(get_completion(messages))

このアプローチは始めるのに素晴らしい方法であり、いくつかのユースケースでは最高のパフォーマンスを提供します。しかし、いくつかの制限があります。
- 最大20枚の画像しか含めることができません（この制限は今後増やす予定です）
- RAGの一部としてスライドコンテンツを使用している場合、埋め込みに画像を導入すると問題が発生する可能性があります

幸いなことに、私たちはClaudeの視覚機能を活用して、通常のpdf転写が許可するよりもはるかに高品質なスライドデッキの表現を**テキスト形式**で得ることができます。

これを行う最良の方法は、Claudeにデッキを最初から最後まで順番にナレーションさせ、現在のスライドとその前のナレーションを渡すことです。では、見てみましょう。

In [ ]:
# スライドデッキをナレーションするためのプロンプトを作成する2つの関数を定義します。デッキの性質に基づいてこれらのプロンプトを調整しますが、構造は大きく変えません。
def build_previous_slides_prompt(previous_slide_narratives):
    prompt = '\n'.join([f"<slide_narration id={index+1}>\n{narrative}\n</slide_narration>" for index, narrative in enumerate(previous_slide_narratives)])
    return prompt

def build_slides_narration_prompt(previous_slide_narratives):
    if len(previous_slide_narratives) == 0:
        prompt = """あなたはTwilioのCFOであり、2023年第4四半期の収益プレゼンテーションをナレーションしています。

現在、スライド1にいます。画像に表示されています。
プレゼンターのようにTwilioの2023年第4四半期の収益プレゼンテーションのこのページをナレーションしてください。意味が正確にわからない場合は、特に略語については話さないでください。このスライドに明示的に表示されていないことについては議論しないでください。後でその資料をカバーする可能性のあるスライドがさらにあります。
視覚障害のある視聴者がいるため、詳細をナレーションしないと、数字を知らないことになりますので、すべての数字をナレーションしてください。

ナレーションを<narration>タグに入れてください。"""

    else:
        prompt = f"""あなたはTwilioのCFOであり、2023年第4四半期の収益プレゼンテーションをナレーションしています。これまでのスライドからのナレーションは以下の通りです：
<previous_slide_narrations>
{build_previous_slides_prompt(previous_slide_narratives)}
</previous_slide_narrations>

現在、スライド{len(previous_slide_narratives)+1}にいます。画像に表示されています。
プレゼンターのようにTwilioの2023年第4四半期の収益プレゼンテーションのこのページをナレーションしてください。これまでのスライドで言ったことを考慮してください。意味が正確にわからない場合は、特に略語については話さないでください。このスライドに明示的に表示されていないことについては議論しないでください。後でその資料をカバーする可能性のあるスライドがさらにあります。
視覚障害のある視聴者がいるため、詳細をナレーションしないと、数字を知らないことになりますので、すべての数字をナレーションしてください。

非常に詳細に説明してください。

ナレーションを<narration>タグに入れてください。"""
    
    return prompt

In [ ]:
# 今、私たちは関数を使用して全体のデッキをナレーションします。これを実行するのに数分かかる場合があります（しばしば最大10分）。
import re
from tqdm import tqdm
previous_slide_narratives = []
for i, pdf_png in tqdm(enumerate(pdf_pngs)):
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": build_slides_narration_prompt(previous_slide_narratives)},
                {"image": {
                    "format": 'jpeg',
                    "source": {"bytes": pdf_png }
                    },
                }
            ]
        }
    ]
    completion = get_completion(messages)


    pattern = r"<narration>(.*?)</narration>"
    match = re.search(pattern, completion.strip(), re.DOTALL)
    if match:
        narration = match.group(1)
    else:
        raise ValueError("ナレーションが利用できません。")
    
    previous_slide_narratives.append(narration)
    # 生成したナレーションを見たい場合は、以下の行のコメントを外してください
    # print(narration)

slide_narration = build_previous_slides_prompt(previous_slide_narratives)

テキストベースのナレーションができたので（完璧ではありませんが、かなり良いです）、このデッキをテキストのみのワークフローで使用する能力があります。ベクター検索を含めて！

最終的な確認として、ナレーションのみのセットアップにいくつかの質問をしてみましょう！

In [ ]:
questions = [
    "What percentage of q4 total revenue was the Segment business line?",
    "Has the rate of growth of quarterly revenue been increasing or decreasing? Give just an answer.",
    "What was acquisition revenue for the year ended december 31, 2023 (including negative revenues)?"
]

for index, question in enumerate(questions):
    prompt = f"""You are an expert financial analyst analyzing a transcript of Twilio's earnings call.
Here is the transcript:
<transcript>
{slide_narration}
</transcript>

Please answer the following question:
<question>
{question}
</question>"""
    messages = [
        {
            "role": 'user',
            "content": [
                {"text": prompt},
            ]
        }
    ]

    print(f"\n----------質問 {index+1}----------")
    print(get_completion(messages))

これらの技術を駆使すれば、スライドデッキのようなチャートやグラフが多いコンテンツにモデルを適用する準備が整います。